In [1]:
import java.io.File
abstract class Problem<T>(private val filename: String) {
    abstract val testResult: T
    protected var isTesting: Boolean = true
        private set
    fun run() {
        val testOutput = File("testData/$filename").useLines { solve(it.toList()) }
        if (testOutput != testResult) {
            throw IllegalStateException("Test output doesn't match expected result \n Actual: $testOutput \n Expected: $testResult")
        }
        println("Test results: Expected=$testResult Actual=$testOutput")

        isTesting = false
        val actualOutput = File("data/$filename").useLines { solve(it.toList()) }
        println("Solution: $actualOutput")
    }
    abstract fun solve(lines: List<String>): T
}

## Day 7: Tachyon Manifold
### Part 1
You thank the cephalopods for the help and exit the trash compactor, finding yourself in the familiar halls of a North Pole research wing.

Based on the large sign that says "teleporter hub", they seem to be researching teleportation; you can't help but try it for yourself and step onto the large yellow teleporter pad.

Suddenly, you find yourself in an unfamiliar room! The room has no doors; the only way out is the teleporter. Unfortunately, the teleporter seems to be leaking magic smoke.

Since this is a teleporter lab, there are lots of spare parts, manuals, and diagnostic equipment lying around. After connecting one of the diagnostic tools, it helpfully displays error code 0H-N0, which apparently means that there's an issue with one of the tachyon manifolds.

You quickly locate a diagram of the tachyon manifold (your puzzle input). A tachyon beam enters the manifold at the location marked S; tachyon beams always move downward. Tachyon beams pass freely through empty space (.). However, if a tachyon beam encounters a splitter (^), the beam is stopped; instead, a new tachyon beam continues from the immediate left and from the immediate right of the splitter.

For example:
```
.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
In this example, the incoming tachyon beam (|) extends downward from S until it reaches the first splitter:
```
.......S.......
.......|.......
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
At that point, the original beam stops, and two new beams are emitted from the splitter:
```
.......S.......
.......|.......
......|^|......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
Those beams continue downward until they reach more splitters:
```
.......S.......
.......|.......
......|^|......
......|.|......
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
At this point, the two splitters create a total of only three tachyon beams, since they are both dumping tachyons into the same place between them:
```
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
This process continues until all of the tachyon beams reach a splitter or exit the manifold:
```
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
```
To repair the teleporter, you first need to understand the beam-splitting properties of the tachyon manifold. In this example, a tachyon beam is split a total of `21` times.

Analyze your manifold diagram. How many times will the beam be split?

In [2]:
class DaySevenPartOne(override val testResult: Int = 21) : Problem<Int>("day7.txt") {
    override fun solve(lines: List<String>): Int {
        var beamPositions = setOf<Int>()
        var numSplits = 0
        for (row in 0 until lines.size) {
            val rowBeamPositions = beamPositions.toMutableSet()
            lines[row].forEachIndexed { col, char ->
                if (char == 'S') {
                    rowBeamPositions.add(col)
                }

                if (char == '^' && beamPositions.contains(col)) {
                    rowBeamPositions.remove(col)
                    rowBeamPositions.addAll(listOf(col - 1, col + 1))
                    numSplits++
                }
            }
            beamPositions = rowBeamPositions
        }
        return numSplits
    }
}

DaySevenPartOne().run()

Test results: Expected=21 Actual=21
Solution: 1660


### Part 2
With your analysis of the manifold complete, you begin fixing the teleporter. However, as you open the side of the teleporter to replace the broken manifold, you are surprised to discover that it isn't a classical tachyon manifold - it's a quantum tachyon manifold.

With a quantum tachyon manifold, only a single tachyon particle is sent through the manifold. A tachyon particle takes both the left and right path of each splitter encountered.

Since this is impossible, the manual recommends the many-worlds interpretation of quantum tachyon splitting: each time a particle reaches a splitter, it's actually time itself which splits. In one timeline, the particle went left, and in the other timeline, the particle went right.

To fix the manifold, what you really need to know is the number of timelines active after a single particle completes all of its possible journeys through the manifold.

In the above example, there are many timelines. For instance, there's the timeline where the particle always went left:
```
.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
...|^.^...^....
...|...........
..|^.^...^.^...
..|............
.|^...^.....^..
.|.............
|^.^.^.^.^...^.
|..............
```
Or, there's the timeline where the particle alternated going left and right at each splitter:
```
.......S.......
.......|.......
......|^.......
......|........
......^|^......
.......|.......
.....^|^.^.....
......|........
....^.^|..^....
.......|.......
...^.^.|.^.^...
.......|.......
..^...^|....^..
.......|.......
.^.^.^|^.^...^.
......|........
```
Or, there's the timeline where the particle ends up at the same point as the alternating timeline, but takes a totally different path to get there:
```
.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
....^|^...^....
.....|.........
...^.^|..^.^...
......|........
..^..|^.....^..
.....|.........
.^.^.^|^.^...^.
......|........
```
In this example, in total, the particle ends up on 40 different timelines.

Apply the many-worlds interpretation of quantum tachyon splitting to your manifold diagram. In total, how many different timelines would a single tachyon particle end up on?

In [3]:
class DaySevenPartTwo(override val testResult: Long = 40) : Problem<Long>("day7.txt") {
    override fun solve(lines: List<String>): Long {
        val data = lines.filterNot { line -> line.all { it == '.' } }
        val start = data.first().indexOfFirst { it == 'S' }
        var paths: Map<Int, Long> = mapOf(start to 1L)
        data.drop(1).forEach { line ->
            val lastPaths = paths.toMap()
            val currentPaths = paths.toMutableMap()
            line.forEachIndexed { index, char ->
                lastPaths[index]?.let {
                    if (char == '^' && it > 0) {
                        currentPaths[index] = 0
                        currentPaths[index - 1] = (lastPaths[index] ?: 0) + (currentPaths[index - 1] ?: 0)
                        currentPaths[index + 1] = (lastPaths[index] ?: 0) + (currentPaths[index + 1] ?: 0)
                    }
                }
            }
            paths = currentPaths
        }

        return paths.entries.sumOf { it.value }
    }
}

DaySevenPartTwo().run()

Test results: Expected=40 Actual=40
Solution: 305999729392659
